In [18]:
import requests
import pandas as pd
import json,os
import requests
import pandas as pd
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
#sharepoint dev site
tenant_id = os.getenv('TENANT_ID')
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

# Token endpoint
url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"

# Get token for Microsoft Graph
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': 'https://graph.microsoft.com/.default'
}

token_response = requests.post(url, data=data)
access_token = token_response.json()['access_token']

veeva_headers = {
    'Authorization': '053C78ACD9A27EA36A32FAAC65F3CE72A78B97F7AA9176ED933313F362FD0024132D77598543F2502C16BF103E3A09D2D35814FABAF972DDA2F6FF59086E5918',  # Add your token
    'Content-Type': 'application/x-www-form-urlencoded',
}
sp_headers = {
        'Authorization': f'Bearer {access_token}',  
        'Content-Type': 'application/octet-stream'
        }
LOCAL_DOWNLOAD_PATH = './'

Upload Testing into Sharepoint

In [ ]:
file_download_url = f"https://sb-envu-deployment.veevavault.com/api/v24.2/objects/documents/25102/file"
file_name = 'Label_Albania_Finnish111.docx'
file_response = requests.get(url=file_download_url,headers=veeva_headers)

print(f"file_response.statuscode:{file_response.status_code}")
drive_id = 'b!wo3sBA26VUyRp2CMZopLbM3kX6cO3U5KvGeckT3yjRGApZ-K82XyQLCxWq_PAK6q'
folder_name = 'Shared Documents'
folder_name_encoded = folder_name.replace(" ", "%20") if folder_name else ""



upload_url  = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root:/{file_name}:/content"
upload_response  = requests.put(upload_url , headers=sp_headers, data=file_response.content)

file_response.statuscode:200


to lists all drives in a site

In [ ]:
update_url = "https://graph.microsoft.com/v1.0/sites/envu.sharepoint.com:/sites/EnvuSpecificationsProd:/drives"

document_info = []
upload_response  = requests.get(update_url , headers=sp_headers)
print(json.dumps(upload_response.json(),indent=3))

To lists all Files and metadata for  a perticular Drive ID

In [ ]:
drive_id = 'b!wo3sBA26VUyRp2CMZopLbM3kX6cO3U5KvGeckT3yjRGApZ-K82XyQLCxWq_PAK6q'
url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root/children"
sp_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}
upload_response  = requests.get(url , headers=sp_headers)
print(json.dumps(upload_response.json(),indent=3))


To list all Fields for a perticular item_id (File ID)

In [ ]:
item_id = '01S4YKXPJFONBX6LMNDJEJVMGVBFB23RIP'
drive_id = 'b!wo3sBA26VUyRp2CMZopLbM3kX6cO3U5KvGeckT3yjRGApZ-K82XyQLCxWq_PAK6q'
fields_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item_id}/listItem/fields"
sp_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}
upload_response  = requests.get(fields_url , headers=sp_headers)
print(json.dumps(upload_response.json(),indent=3))

Excel Preparation

In [ ]:
import requests
import pandas as pd
import json

drive_id = 'b!wo3sBA26VUyRp2CMZopLbM3kX6cO3U5KvGeckT3yjRGApZ-K82XyQLCxWq_PAK6q'
url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root/children"
sp_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

document_info = []

while url:
    response = requests.get(url, headers=sp_headers).json()
    items = response.get('value', [])
    
    for record in items:
        internal_record = {
            'id': record.get('id'),
            'filename': record.get('name'),
            'createdDateTime': record.get('createdDateTime'),
            'lastModifiedDateTime': record.get('lastModifiedDateTime'),
            'webUrl': record.get('webUrl')
        }
        document_info.append(internal_record)
    
    # Check for next page link
    url = response.get('@odata.nextLink')

# Convert to DataFrame
document_df = pd.DataFrame(document_info)
document_df = document_df[['id','webUrl']]

print(f"✅ Total documents fetched: {len(document_df)}")
document_df

MetaData of each file for above id's and weburl's

In [ ]:
# Your existing document DataFrame with 'id' column
ids = document_df['id'].tolist()

drive_id = 'b!wo3sBA26VUyRp2CMZopLbM3kX6cO3U5KvGeckT3yjRGApZ-K82XyQLCxWq_PAK6q'
sp_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# Function to fetch one file’s metadata
def fetch_fields(item_id):
    fields_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item_id}/listItem/fields"
    try:
        response = requests.get(fields_url, headers=sp_headers, timeout=10)
        if response.status_code == 200:
            fields = response.json()
            return {
                'id': item_id,
                'SpecificationNumber': fields.get('SpecificationNumber'),
                'filename': fields.get('FileName'),
                'DocumentType': fields.get('DocumentType'),
                'DocumentDescription': fields.get('DocumentDescription'),
                'DocumentStatus': fields.get('DocumentStatus')
            }
        else:
            return {'id': item_id, 'error': response.status_code}
    except Exception as e:
        return {'id': item_id, 'error': str(e)}

# Use a thread pool for concurrency
meta_data = []
with ThreadPoolExecutor(max_workers=20) as executor:  # You can tune max_workers (10–30 works well)
    futures = {executor.submit(fetch_fields, i): i for i in ids}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Fetching metadata"):
        meta_data.append(future.result())

# Convert to DataFrame
meta_data_df = pd.DataFrame(meta_data)
print(meta_data_df.head())


In [ ]:
merge_df = pd.merge(document_df,meta_data_df,on='id',how='left')
merge_df = merge_df[['SpecificationNumber','filename','DocumentType','DocumentDescription','DocumentStatus','webUrl']]
merge_df.rename(columns={'filename': 'FileName', 'webUrl': 'Document URL'}, inplace=True)
merge_df.to_excel('SharepointLinkUpdateUAT.xlsx',index=False)